<a href="https://colab.research.google.com/github/wanutchapornmun/Statistical-Learning-Labs/blob/main/229351_LAB06_660510561.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ปฏิบัติการครั้งที่ 5 กระบวนวิชา 229351 Statistical Learning for Data Science

คำชี้แจง

ให้เริ่มทำปฏิบัติการจาก colab notebook ที่กำหนดให้ จากนั้น share แล้วส่ง link มาใน mango.cmu.ac.th

ดาวน์โหลดข้อมูลรถยนต์ชนิดต่างใน link ข้างล่างนี้
https://donlapark.pages.dev/229351/data/Auto.csv
หรือรันโค้ดข้างล่าง

In [3]:
!wget https://donlapark.pages.dev/229351/data/Auto.csv

--2025-08-07 05:46:53--  https://donlapark.pages.dev/229351/data/Auto.csv
Resolving donlapark.pages.dev (donlapark.pages.dev)... 172.66.47.56, 172.66.44.200, 2606:4700:310c::ac42:2f38, ...
Connecting to donlapark.pages.dev (donlapark.pages.dev)|172.66.47.56|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18079 (18K) [text/csv]
Saving to: ‘Auto.csv’

Auto.csv            100%[===================>]  17.66K  --.-KB/s    in 0s      

2025-08-07 05:46:53 (81.1 MB/s) - ‘Auto.csv’ saved [18079/18079]



In [4]:
import numpy as np
import pandas as pd
from scipy import stats

In [5]:
# อ่านไฟล์ csv ก่อนเพื่อหา missing values
auto_df = pd.read_csv('Auto.csv',na_values=["?"])
predictor_names = auto_df.columns

# ลบแถวที่มี missing values
auto_df = auto_df.dropna()
auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year,origin,name
0,18.0,8,307.0,130.0,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449,10.5,70,1,ford torino


In [6]:
predictor_names

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'year', 'origin', 'name'],
      dtype='object')

### ตัวแปรต่าง ๆ มีดังนี้
mpg

    miles per gallon
cylinders

    Number of cylinders between 4 and 8
displacement

    Engine displacement (cu. inches)
horsepower

    Engine horsepower
weight

    Vehicle weight (lbs.)
acceleration

    Time to accelerate from 0 to 60 mph (sec.)
year

    Model year (modulo 100)
origin

    Origin of car (1. American, 2. European, 3. Japanese)
name

    Vehicle name


In [7]:
auto_df = auto_df.drop(['name', 'origin'],axis=1)

auto_df.head()

,mpg,cylinders,displacement,horsepower,weight,acceleration,year
0,18.0,8,307.0,130.0,3504,12.0,70
1,15.0,8,350.0,165.0,3693,11.5,70
2,18.0,8,318.0,150.0,3436,11.0,70
3,16.0,8,304.0,150.0,3433,12.0,70
4,17.0,8,302.0,140.0,3449,10.5,70


## 1. Multivariate linear regression

เปลี่ยน $\texttt{dataframe}$ ให้เป็น numpy array ด้วย `dataframe.to_numpy()`

1. Response: $\texttt{y}$ เป็นคอลัมน์เวกเตอร์ของ `mpg`
2. Predictors: $\texttt{X}$ เป็นเมทริกซ์ของตัวแปรที่เหลือ (ใช้ `auto_df.drop`)


In [8]:
y = auto_df['mpg'].to_numpy()
X = auto_df.drop(['mpg'],axis=1).to_numpy()

คอลัมน์แรกของ $\texttt{X}$ ต้องเป็น $(1, 1, \ldots ,1 )^T$ (จำนวนเลข 1 เท่ากับจำนวนแถวของ $\texttt{X}$)  
สร้างเวกเตอร์ที่มีแต่เลข 1 และมีความยาวที่เหมาะสมด้วยคำสั่งใดคำสั่งหนึ่งต่อไปนี้  
1. $\texttt{[1]*X.shape[0]}$  
2. $\texttt{np.ones(X.shape[0])}$  

In [9]:
X.shape

(392, 6)

In [10]:
ones =  [1]*X.shape[0] #your answer here

# ใส่ ones ให้เป็นคอลัมน์แรกของ X
X = np.c_[ones,X]

In [11]:
#เช็คคำตอบ
print(X)

[[1.000e+00 8.000e+00 3.070e+02 ... 3.504e+03 1.200e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.500e+02 ... 3.693e+03 1.150e+01 7.000e+01]
 [1.000e+00 8.000e+00 3.180e+02 ... 3.436e+03 1.100e+01 7.000e+01]
 ...
 [1.000e+00 4.000e+00 1.350e+02 ... 2.295e+03 1.160e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.200e+02 ... 2.625e+03 1.860e+01 8.200e+01]
 [1.000e+00 4.000e+00 1.190e+02 ... 2.720e+03 1.940e+01 8.200e+01]]


### Exercise 1: จงเขียนฟังก์ชันต่อไปนี้
1. `linear_model` มี input เป็นแมทริกซ์ `X` เวกเตอร์ `y` และ output เป็นเวกเตอร์ของสัมประสิทธิ์
2. `predict` มี input เป็นแมทริกซ์ `X` เวกเตอร์ของสัมประสิทธิ์ `beta` และ output เป็นเวกเตอร์ของค่าทำนาย `y_pred`

1. Matrix multiplication $AB = \texttt{A@B}$
2. Transpose: $X^T = \texttt{X.T}$
3. Inverse: $X^{-1} = \texttt{np.linalg.inv(X)}$

$$ \hat{\beta} = (X^TX)^{-1}X^Ty $$

In [12]:
# Let's start by implementing linear regression from scratch
# using the formula provided in the class
import numpy as np
def linear_model(X, y):
    """X: เมทริกซ์ของตัวแปรต้น"""
    """y: เวกเตอร์ของตัวแปรตาม"""
    """Return: เวกเตอร์ของ parameter beta จากการสร้าง linear regressions model
    ด้วย Ordinary Least Squares (OLS)"""
    # TODO: your code here
    beta = np.linalg.inv(X.T@X)@X.T@y


    return beta


def predict(X, beta):
    """beta: array ของสัมประสิทธิ์"""
    """X: เมทริกซ์ของตัวแปรต้น"""
    """Return: ค่าทำนาย y_pred = X*beta """
    # TODO: your code here
    y_pred = X @ beta


    return y_pred

In [13]:
beta = linear_model(X, y)
y_pred = predict(X, beta)

print("Coefficients are:", beta)
print("Predictions are:", y_pred)

Coefficients are: [-1.45352505e+01 -3.29859089e-01  7.67843024e-03 -3.91355574e-04
 -6.79461791e-03  8.52732469e-02  7.53367180e-01]
Predictions are: [15.08291904 14.07257469 15.53631544 15.53447451 15.28640745 10.13543367
 10.14518132 10.2823774   9.75375835 13.04735326 15.07939657 14.27003871
 13.82978602 18.85175174 22.8737646  19.77703076 20.18480893 21.5104138
 24.35531011 26.88779881 21.02865789 22.39292633 22.99745799 23.65262179
 21.00103365  8.07833662  9.38643707  9.29830867  7.24571366 25.10867729
 24.61286153 24.52028351 21.92845363 16.61626354 17.55758904 17.74574
 17.69795664 11.36255565  9.96729942 11.8765442  11.97578129  6.49865193
  8.09581057  5.41677433 19.93818121 23.93993731 17.83429946 18.76798947
 24.64726853 25.25871046 25.78437479 25.48591785 27.72744749 28.73542612
 27.51457176 26.77034873 25.06182966 26.10937612 25.80027716 24.72889672
 25.57305112 11.67427267 11.30007803 12.50679499 12.75714021 15.37465857
  9.73949929 10.25692321 10.56751885 11.08544346 24

#### จงหา $n$ (จำนวนแถวของ $\texttt{X}$) และ $p$ (จำนวนตัวแปร) ด้วย method $\texttt{X.shape[...]}$

In [14]:
n = X.shape[0]

p = X.shape[1]-1

print('Number of observations:', n)
print('Number of variables:', p)

Number of observations: 392
Number of variables: 6


## 2. F-statistic

### Exercise 2: เขียนสมมติฐานหลัก ($H_0$) และสมมติฐานแย้ง ($H_a$) ที่ใช้ F-statistic ในการทดสอบ

$H_0$ : ${\beta}_1$ = ${\beta}_2$ = ${\beta}_3$ = ${\beta}_4$ = ${\beta}_5$ = ${\beta}_6$ = 0

$H_a$ : at least one of ${\beta}_1$, ${\beta}_2$, ${\beta}_3$, ${\beta}_4$, ${\beta}_5$, ${\beta}_6$, ${\beta}_6$ is non-zero.

### Exercise 3: จากเวกเตอร์ของ outcomes `y` คำนวณ Total Sum of Squares (TSS)

$$ \text{TSS} = \sum_{i=1}^n (y_i-\bar{y})^2 $$

$$ \text{RSS} = \sum_{i=1}^n (y_i-\hat{y})^2 $$
โดยใช้คำสั่งต่อไปนี้

$\texttt{np.sum}([y_1,y_2,...,y_n])=y_1+y_2+...+y_n$

$\texttt{np.square}([y_1,y_2,...,y_n])=[y_1^2,y^2_2,...,y^2_n]$

In [15]:
# Calculate Total Sum of Squares
y_mean = y.mean()

TSS = np.sum(np.square(y-y_mean))# TODO: your code here

RSS = np.sum(np.square(y-y_pred))

คำนวณ $F$-statistic ด้วย
$$F_{p,n-p-1} = \frac{(\text{TSS}-\text{RSS})/p}{\text{RSS}/(n-p-1)}$$
หา $n$ และจำนวนของตัวแปร $p$ ด้วย $\texttt{X.shape}[\cdot]$

### Exercise 4: คำนวณค่า F

In [16]:
F = ((TSS - RSS)/p)/(RSS/(n - p -1))

print(F)

272.23409822574587


In [17]:
# Compute p_value of the F-statistic

p_value = stats.f.sf(F, p, n-p-1)

print(p_value)

3.792336100722653e-135


### Exercise 5: จากการทดสอบสมมติฐานที่นัยสำคัญ 0.05 เราได้ข้อสรุปอย่างไรจากค่า p-value ที่ได้ข้างบน

### คำตอบของ Exercise 5:




The F-value is 272.23409822574587 with its corresponding p-value = 3.792336100722653e-135

We are certain that at least one of 'cylinders', 'displacement', 'horsepower', 'weight', 'acceleration', 'year' must be related to the 'mpg'

## 3. คำนวณ $t$-statistic

#### ขั้นตอนในการคำนวณ $t$-statistic

\begin{align*}
\text{RSE} &= \sqrt{\frac{\text{RSS}}{n-p-1}} \\
\text{SE}(\hat{\beta}) &=\text{RSE}\cdot\sqrt{\text{diag}\left( (X^TX)^{-1}\right)} \\
t &= \frac{\hat{\beta}}{\text{SE}(\hat{\beta})}
\end{align*}

In [18]:
np.array([[1, 2], [3, 4]]).diagonal() # ดึงค่าในแนวทแยงของ matrix นี้ออกมา

array([1, 4])

In [19]:
def t_to_p_values(t_statistics):
    """t_statistics: numpy array ของค่า t-statistic ของตัวแปรต่างๆ
    Return: เวกเตอร์ของค่า p-value ของค่า t-statistic ที่ให้มา"""
    return stats.t.sf(np.abs(t_statistics), n-p-1)*2

# Standard error of each variable in X
RSE = np.sqrt(RSS/(n-p-1))
SE = RSE * np.sqrt((np.linalg.inv(X.T@X).diagonal()))
t = beta  / SE
p = t_to_p_values(t)

In [20]:
print(RSE)
print(SE)
print(t)
print(p)

3.435244003399977
[4.76388189e+00 3.32104133e-01 7.35773614e-03 1.38365215e-02
 6.70022695e-04 1.02035567e-01 5.26181480e-02]
[ -3.05113578  -0.99323994   1.04358597  -0.02828425 -10.14087727
   0.83572081  14.3176301 ]
[2.43774108e-03 3.21216868e-01 2.97331805e-01 9.77450101e-01
 1.41618989e-21 4.03830330e-01 1.41042759e-37]


### Exercise 6: จงใช้ฟังก์ชัน `t_to_p_values` ในการหาว่ามีตัวแปรใดบ้างที่มีค่า p-value น้อยกว่า 0.05 **แล้วระบุตัวแปรทั้งหมดใน text block ข้างล่าง**  

#### ใช้คำสั่ง `A.diagonal()` เพื่อดึงค่าในแนวทแยงของ `A`

In [21]:
# TODO: your code here
#RSE =  RSE # scalar
#SE = # vector
#t = # vector

In [22]:
for i in range(len(p)):
    if p[i] < 0.05:
        print(predictor_names[i])

mpg
weight
year


list ชื่อตัวแปร

In [23]:
print("List of predictors:", predictor_names[1:-2])

List of predictors: Index(['cylinders', 'displacement', 'horsepower', 'weight', 'acceleration',
       'year'],
      dtype='object')


### คำตอบของ Exercise 6:





ตัวแปรที่มีค่า p-value น้อยกว่า 0.05 คือ

1.   weight

2.   year



## 4. Variable selection

#### statsmodels library สามารถคำนวณค่าต่างๆ เหล่านี้ได้ ผลที่แสดงจะคล้ายกับใน R  
#### มีสองวิธึในการทำ linear regression: $\texttt{statsmodels.api.sm.OLS}$
#### และ $\texttt{statsmodels.formula.api.smf.ols}$

In [24]:
import statsmodels.api as sm

X = auto_df.drop(['mpg'], axis=1) # matrix นี้ยังไม่มีเลข 1
X = sm.add_constant(X)     # add bias constant คำสั่งนี้คือใส่เลข 1 เข้าไปใน matrix
y = auto_df['mpg']

model = sm.OLS(y, X).fit()
print(model.summary()) # ตัวแปรไหนที่มีค่า p เยอะๆ จะต้องเอาออกก่อน

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 07 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        05:46:54   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const          -14.5353      4.764     -3.051   

In [25]:
import statsmodels.formula.api as smf

model2 = smf.ols('mpg ~ cylinders + displacement + horsepower + weight + acceleration + year', auto_df)
model2 = model2.fit()
print(model2.summary())

                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.809
Model:                            OLS   Adj. R-squared:                  0.806
Method:                 Least Squares   F-statistic:                     272.2
Date:                Thu, 07 Aug 2025   Prob (F-statistic):          3.79e-135
Time:                        05:46:54   Log-Likelihood:                -1036.5
No. Observations:                 392   AIC:                             2087.
Df Residuals:                     385   BIC:                             2115.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept      -14.5353      4.764     -3.051   

### Exercise 7: ใช้ Backward selection ในการเลือกตัวแปร โดยที่เราจะหยุดเมื่อ p-value สของตัวแปรทุกตัว < 0.05 แล้วเขียนสมการของโมเดลที่ได้

In [31]:
# Block for running your own code
X = auto_df.drop(['mpg'], axis=1)
y = auto_df['mpg']

while True:
    X_const = sm.add_constant(X)
    model = sm.OLS(y, X_const).fit()
    pvalues = model.pvalues.drop('const')

    p_max = pvalues.max()
    if p_max > 0.05:
        var_to_remove = pvalues.idxmax()
        X = X.drop(columns=[var_to_remove])
    else:
        break

print(model.summary())


                            OLS Regression Results                            
Dep. Variable:                    mpg   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     819.5
Date:                Thu, 07 Aug 2025   Prob (F-statistic):          3.33e-140
Time:                        05:56:10   Log-Likelihood:                -1037.6
No. Observations:                 392   AIC:                             2081.
Df Residuals:                     389   BIC:                             2093.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -14.3473      4.007     -3.581      0.0

In [27]:
# If you want to write with for-loop
# อยากดึงค่า t ออกมา
print(model2.tvalues)
print("t-value of weight:", model2.tvalues["weight"])
# column ที่ 2 คือ ค่า t value

Intercept       -3.051136
cylinders       -0.993240
displacement     1.043586
horsepower      -0.028284
weight         -10.140877
acceleration     0.835721
year            14.317630
dtype: float64
t-value of weight: -10.140877265267768


In [28]:
# If you want to write with for-loop
# อยากดึงค่า p ออกมา
print(model2.pvalues)
print("p-value of weight:", model2.pvalues["weight"])
# column ที่ 2 คือ ค่า p value
# ทำไปเรื่อย ๆ จน ค่า p value เป็น 0.05

Intercept       2.437741e-03
cylinders       3.212169e-01
displacement    2.973318e-01
horsepower      9.774501e-01
weight          1.416190e-21
acceleration    4.038303e-01
year            1.410428e-37
dtype: float64
p-value of weight: 1.416189889352751e-21


### คำตอบของ Exercise 7:

$$
y = -14.5353 - 0.0066*\text{weight} + 0.7573 *\text{year}
$$

In [32]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    mpg   R-squared:                       0.808
Model:                            OLS   Adj. R-squared:                  0.807
Method:                 Least Squares   F-statistic:                     819.5
Date:                Thu, 07 Aug 2025   Prob (F-statistic):          3.33e-140
Time:                        06:03:10   Log-Likelihood:                -1037.6
No. Observations:                 392   AIC:                             2081.
Df Residuals:                     389   BIC:                             2093.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -14.3473      4.007     -3.581      0.000     -22.224      -6.470
weight        -0.0066      0.000    -30.911      0.000      -0.007      -0.006
year           0.7573      0.049     15.308      0.000       0.660       0.855
==============================================================================
Omnibus:                       42.504   Durbin-Watson:                   1.230
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               71.997
Skew:                           0.670   Prob(JB):                     2.32e-16
Kurtosis:                       4.616   Cond. No.                     7.17e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""